In [ ]:
%pip install --upgrade --quiet transformers

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "NousResearch/Hermes-3-Llama-3.1-8B"
#model_path = "NousResearch/Hermes-2-Pro-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/interstellarninja/miniforge3/envs/hermes-tools/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
from pydantic import BaseModel
from typing import Any
import re
import json

class FunctionCall(BaseModel):
    name: str
    arguments: dict[str, Any]

In [3]:
print(FunctionCall.schema_json())

{"properties": {"name": {"title": "Name", "type": "string"}, "arguments": {"title": "Arguments", "type": "object"}}, "required": ["name", "arguments"], "title": "FunctionCall", "type": "object"}


In [4]:
template_str = """{%- macro json_to_python_type(json_spec) %}
{%- set basic_type_map = {
    "string": "str",
    "number": "float",
    "integer": "int",
    "boolean": "bool"
} %}

{%- if basic_type_map[json_spec.type] is defined %}
    {{- basic_type_map[json_spec.type] }}
{%- elif json_spec.type == "array" %}
    {{- "list[" +  json_to_python_type(json_spec|items) + "]"}}
{%- elif json_spec.type == "object" %}
    {%- if json_spec.additionalProperties is defined %}
        {{- "dict[str, " + json_to_python_type(json_spec.additionalProperties) + ']'}}
    {%- else %}
        {{- "dict" }}
    {%- endif %}
{%- elif json_spec.type is iterable %}
    {{- "Union[" }}
    {%- for t in json_spec.type %}
      {{- json_to_python_type({"type": t}) }}
      {%- if not loop.last %}
        {{- "," }} 
    {%- endif %}
    {%- endfor %}
    {{- "]" }}
{%- else %}
    {{- "Any" }}
{%- endif %}
{%- endmacro %}


{{- bos_token }}
{{- '<|im_start|>system
' }}
{{- "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> " }}
{%- for tool in tools %}
    {%- if tool.function is defined %}
        {%- set tool = tool.function %}
    {%- endif %}
    {{- '{"type": "function", "function": ' }}
    {{- '{"name": "' + tool.name + '", ' }}
    {{- '"description": "' + tool.name + '(' }}
    {%- for param_name, param_fields in tool.parameters.properties|items %}
        {{- param_name + ": " + json_to_python_type(param_fields) }}
        {%- if not loop.last %}
            {{- ", " }}
        {%- endif %}
    {%- endfor %}
    {{- ")" }}
    {%- if tool.return is defined %}
        {{- " -> " + json_to_python_type(tool.return) }}
    {%- endif %}
    {{- " - " + tool.description + "\n\n" }}
    {%- for param_name, param_fields in tool.parameters.properties|items %}
        {%- if loop.first %}
            {{- "    Args:\n" }}
        {%- endif %}
        {{- "        " + param_name + "(" + json_to_python_type(param_fields) + "): " + param_fields.description|trim }}
    {%- endfor %}
    {%- if tool.return is defined and tool.return.description is defined %}
        {{- "\n    Returns:\n        " + tool.return.description }}
    {%- endif %}
    {{- '"' }}
    {{- ', "parameters": ' }}
    {%- if tool.parameters.properties | length == 0 %}
        {{- "{}" }}
    {%- else %}
        {{- tool.parameters|tojson }}
    {%- endif %}
    {{- "}" }}
    {%- if not loop.last %}
        {{- "\n" }}
    {%- endif %}
{%- endfor %}
{{- " </tools>" }}
{{- 'Use the following pydantic model json schema for each tool call you will make: {"properties": {"name": {"title": "Name", "type": "string"}, "arguments": {"title": "Arguments", "type": "object"}}, "required": ["name", "arguments"], "title": "FunctionCall", "type": "object"}}
' }}
{{- "For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
" }}
{{- "<tool_call>
" }}
{{- '{"name": <function-name>, "arguments": <args-dict>}
' }}
{{- '</tool_call><|im_end|>\n' }}
{%- for message in messages %}
    {%- if message.role == "user" or message.role == "system" or (message.role == "assistant" and message.tool_calls is not defined) %}
        {{- '<|im_start|>' + message.role + '\n' + message.content + '<|im_end|>' + '\n' }}
    {%- elif message.role == "assistant" %}
        {{- '<|im_start|>' + message.role }}
    {%- for tool_call in message.tool_calls %}
       {{- '
<tool_call>
' }}           {%- if tool_call.function is defined %}
                {%- set tool_call = tool_call.function %}
            {%- endif %}
            {{- '{' }}
            {{- '"name": "' }}
            {{- tool_call.name }}
            {{- '"' }}
            {{- ', '}}
            {%- if tool_call.arguments is defined %}
                {{- '"arguments": ' }}
                {%- if tool_call.arguments is string %}
                    {{- tool_call.arguments }}
                {%- else %}
                    {{- tool_call.arguments|tojson }}
                {%- endif %}
            {%- endif %}
             {{- '}' }}
            {{- '\n</tool_call>' }}
    {%- endfor %}
        {{- '<|im_end|>\n' }}
    {%- elif message.role == "tool" %}
        {%- if loop.previtem and loop.previtem.role != "tool" %}
            {{- '<|im_start|>tool\n' }}
        {%- endif %}
        {{- '<tool_response>\n' }}
        {{- message.content }}
        {%- if not loop.last %}
            {{- '\n</tool_response>\n' }}
        {%- else %}
            {{- '\n</tool_response>' }}
        {%- endif %}
        {%- if not loop.last and loop.nextitem.role != "tool" %}
            {{- '<|im_end|>' }}
        {%- elif loop.last %}
            {{- '<|im_end|>' }}
        {%- endif %}
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|im_start|>assistant\n' }}
{%- endif %}
"""

In [5]:
import json

# Assuming template_str is your original multi-line string
escaped_template = json.dumps(template_str)

# Create the final formatted string
formatted_template = f'"template": {escaped_template}'

print(formatted_template)

"template": "{%- macro json_to_python_type(json_spec) %}\n{%- set basic_type_map = {\n    \"string\": \"str\",\n    \"number\": \"float\",\n    \"integer\": \"int\",\n    \"boolean\": \"bool\"\n} %}\n\n{%- if basic_type_map[json_spec.type] is defined %}\n    {{- basic_type_map[json_spec.type] }}\n{%- elif json_spec.type == \"array\" %}\n    {{- \"list[\" +  json_to_python_type(json_spec|items) + \"]\"}}\n{%- elif json_spec.type == \"object\" %}\n    {%- if json_spec.additionalProperties is defined %}\n        {{- \"dict[str, \" + json_to_python_type(json_spec.additionalProperties) + ']'}}\n    {%- else %}\n        {{- \"dict\" }}\n    {%- endif %}\n{%- elif json_spec.type is iterable %}\n    {{- \"Union[\" }}\n    {%- for t in json_spec.type %}\n      {{- json_to_python_type({\"type\": t}) }}\n      {%- if not loop.last %}\n        {{- \",\" }} \n    {%- endif %}\n    {%- endfor %}\n    {{- \"]\" }}\n{%- else %}\n    {{- \"Any\" }}\n{%- endif %}\n{%- endmacro %}\n\n\n{{- bos_token }}\n

In [6]:
print(tokenizer.chat_template["tool_use"])

{%- macro json_to_python_type(json_spec) %}
{%- set basic_type_map = {
    "string": "str",
    "number": "float",
    "integer": "int",
    "boolean": "bool"
} %}

{%- if basic_type_map[json_spec.type] is defined %}
    {{- basic_type_map[json_spec.type] }}
{%- elif json_spec.type == "array" %}
    {{- "list[" +  json_to_python_type(json_spec|items) + "]"}}
{%- elif json_spec.type == "object" %}
    {%- if json_spec.additionalProperties is defined %}
        {{- "dict[str, " + json_to_python_type(json_spec.additionalProperties) + ']'}}
    {%- else %}
        {{- "dict" }}
    {%- endif %}
{%- elif json_spec.type is iterable %}
    {{- "Union[" }}
    {%- for t in json_spec.type %}
      {{- json_to_python_type({"type": t}) }}
      {%- if not loop.last %}
        {{- "," }} 
    {%- endif %}
    {%- endfor %}
    {{- "]" }}
{%- else %}
    {{- "Any" }}
{%- endif %}
{%- endmacro %}


{{- bos_token }}
{{- '<|im_start|>system
' }}
{{- "You are a function calling AI model. You are provid

In [7]:
from jinja2 import Template
# The message array
messages = [
    {
        "role": "user",
        "content": "Get the current weather forecast for Paris and get stock price of Hermes International"
    },
    {
        "role": "assistant",
        "tool_calls": [
            {
                "type": "function",
                "function": {
                    "name": "get_weather_forecast",
                    "arguments": "{\"location\": \"Paris\"}"
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_stock_price",
                    "arguments": "{\"symbol\": \"RMS.PA\"}"
                }
            }
        ]
    },
    {
        "role": "tool",
        "content": "{\"name\": \"get_weather_forecast\", \"content\": {\"location\": \"Paris\", \"forecast\": \"Sunny\", \"temperature\": \"+84\\u00b0F\"}}",
        #"tool_call_id": "call_62136354"
        #"name": "get_weather_forecast"
    },
    {
        "role": "tool",
        "content": "{\"name\": \"get_stock_price\", \"content\": \"Unable to fetch stock price for RMS.PA. Response: {'Global Quote': {}}\"}",
        #"tool_call_id": "call_62136355"
        #"name": "get_stock_price"
    }
]

# Create the Jinja2 template
template = Template(template_str)
#template = Template(tokenizer.chat_template["tool_use"])

# Render the template
result = template.render(messages=messages)

# Print the result
print(result)

<|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools>  </tools>Use the following pydantic model json schema for each tool call you will make: {"properties": {"name": {"title": "Name", "type": "string"}, "arguments": {"title": "Arguments", "type": "object"}}, "required": ["name", "arguments"], "title": "FunctionCall", "type": "object"}}
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{"name": <function-name>, "arguments": <args-dict>}
</tool_call><|im_end|>
<|im_start|>user
Get the current weather forecast for Paris and get stock price of Hermes International<|im_end|>
<|im_start|>assistant
<tool_call>
{"name": "get_weather_forecast", "arguments": {"loca

In [8]:
def parse_tool_calls(text):
    pattern = r'<tool_call>\s*(.*?)\s*</tool_call>'
    matches = re.findall(pattern, text, re.DOTALL)
    
    function_calls = []
    for match in matches:
        print(f"Raw match: {match}")  # Print raw match
        try:
            # Remove newlines and extra whitespace
            cleaned_match = re.sub(r'\s+', ' ', match).strip()
            print(f"Cleaned match: {cleaned_match}")  # Print cleaned match
            
            data = json.loads(cleaned_match)
            print(f"Parsed JSON: {data}")  # Print parsed JSON
            
            # Parse the arguments string into a dictionary
            if isinstance(data['arguments'], str):
                data['arguments'] = json.loads(data['arguments'])
            
            function_call = FunctionCall(**data)
            function_calls.append(function_call)
            print(f"Validated function call: {function_call}")  # Print validated function call
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except ValueError as e:
            print(f"Error validating data: {e}")
    
    return function_calls


In [9]:
# Test the updated function
result = parse_tool_calls("""
<|im_start|>assistant
<tool_call>
{"name": "get_weather_forecast", "arguments": {"location": "Paris"}
</tool_call>
<tool_call>
{"name": "get_stock_price", "arguments": {"symbol": "RMS.PA"}
</tool_call><|im_end|>
""")

print("\nFinal result:")
for call in result:
    print(call)

Raw match: {"name": "get_weather_forecast", "arguments": {"location": "Paris"}
Cleaned match: {"name": "get_weather_forecast", "arguments": {"location": "Paris"}
Error parsing JSON: Expecting ',' delimiter: line 1 column 68 (char 67)
Raw match: {"name": "get_stock_price", "arguments": {"symbol": "RMS.PA"}
Cleaned match: {"name": "get_stock_price", "arguments": {"symbol": "RMS.PA"}
Error parsing JSON: Expecting ',' delimiter: line 1 column 62 (char 61)

Final result:
